In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

In [2]:
frame = pd.read_csv('AUCs.txt', sep = '\t', header = 0, parse_dates = [2])
frame.ix[:,2]=frame.ix[:,2].apply(np.float64)

In [3]:
columnames = frame.columns.values[1:]
comb = combinations(columnames, 2)
data_pd = []
for pair in comb:
    data_pd.append((pair,stats.wilcoxon(frame[pair[0]].values-frame[pair[1]].values).pvalue))

In [4]:
cobd_CLS = pd.DataFrame.from_records(data_pd)
cobd_CLS.columns=['name','p']

In [5]:
cobd_CLS

,name,p
0,"(C4.5, C4.5+m)",0.009633
1,"(C4.5, C4.5+cf)",0.861262
2,"(C4.5, C4.5+m+cf)",0.015906
3,"(C4.5+m, C4.5+cf)",0.050301
4,"(C4.5+m, C4.5+m+cf)",0.401534
5,"(C4.5+cf, C4.5+m+cf)",0.022909


In [24]:
cobd_CLS.ix[cobd_CLS.p.argmin()]

name           (C4.5, C4.5+m)
p                  0.00963298
reject                   True
p_corrected         0.0458182
Name: 0, dtype: object

In [6]:
cobd_CLS[cobd_CLS.p<0.05]

,name,p
0,"(C4.5, C4.5+m)",0.009633
2,"(C4.5, C4.5+m+cf)",0.015906
5,"(C4.5+cf, C4.5+m+cf)",0.022909


In [7]:
reject, p_corrected, a1, a2 = multipletests(cobd_CLS.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [8]:
cobd_CLS['reject'] = reject
cobd_CLS['p_corrected'] = p_corrected

In [9]:
cobd_CLS

,name,p,reject,p_corrected
0,"(C4.5, C4.5+m)",0.009633,False,0.057798
1,"(C4.5, C4.5+cf)",0.861262,False,0.861262
2,"(C4.5, C4.5+m+cf)",0.015906,False,0.079532
3,"(C4.5+m, C4.5+cf)",0.050301,False,0.150903
4,"(C4.5+m, C4.5+m+cf)",0.401534,False,0.803069
5,"(C4.5+cf, C4.5+m+cf)",0.022909,False,0.091636


In [10]:
reject, p_corrected, a1, a2 = multipletests(cobd_CLS.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [11]:
cobd_CLS['reject'] = reject
cobd_CLS['p_corrected'] = p_corrected

In [12]:
cobd_CLS

,name,p,reject,p_corrected
0,"(C4.5, C4.5+m)",0.009633,True,0.045818
1,"(C4.5, C4.5+cf)",0.861262,False,0.861262
2,"(C4.5, C4.5+m+cf)",0.015906,True,0.045818
3,"(C4.5+m, C4.5+cf)",0.050301,False,0.075451
4,"(C4.5+m, C4.5+m+cf)",0.401534,False,0.481841
5,"(C4.5+cf, C4.5+m+cf)",0.022909,True,0.045818


In [13]:
cobd_CLS.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64